In [54]:
import warnings
warnings.filterwarnings('ignore')

## Loading & Writing data functions

In [ ]:
import json

In [ ]:
def load_data(filename):
  with open(filename,'r',encoding = 'utf-8') as f:
    data = json.load(f)
  return data

def write_data(filename,data):
  with open(filename,'w',encoding = 'utf-8') as f:
    json.dump(filename,data,indent = 4)

In [ ]:
data = load_data('ushmm_dn.json')['texts']
data[0]

' My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we were very hard working, six children, father and mother and we had a small mill, flour, buckwheat. We were not prosperous but comfortable.  I went to two schools. One was a public school in the morning. In the afternoon I went to a religious school until almost late at night.  Yes.  Well, I raised in the spirit of Judaism.  No, the school itself, in this little city, was segregated between Catholics and Jews. Mind you, it was a small town, and I would say the majority of the people in that small town were Jewish people. Inside the town, somehow, I don\'t know why, but they separated us Jewish children and Catholic children. As you know, most of the people in Poland were Catholic.  Yes, I used to have friends.  Yes, I did. I felt it, maybe not personally, but I knew of a lot of incidents whereby either they were small little -- I would call it -- we were separated, in other words, but ha

## Text Preprocessing

In [ ]:
# nltk
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn # for pos

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Funcitions for text preprocessing

In [ ]:
import re
from collections import defaultdict
from bs4 import BeautifulSoup

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') # removes alphaneumerics
REPLACE_NUMBERS = re.compile('\d')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
 '''
    This function takes raw text as input and returns texts with stopwords,symbols,pucntuations,html tags removed
 '''
 text = BeautifulSoup(text, "lxml").text # HTML decoding
 text = text.lower() # lowercase text
 text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
 text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
 text = REPLACE_NUMBERS.sub('',text) # replace numbers
 text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
 return text

def stemming(sentence):
 
 stemmer = PorterStemmer()
 stemmed = [stemmer.stem(word) for word in sentence.split()]
 #stemSentence = stemSentence.strip()
 return ' '.join(stemmed)


# Lemmatizer function
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lmtzr = WordNetLemmatizer()

def lemmatize(sentence):
  lemmatized_sentence = []
  tokens = word_tokenize(sentence)
  for token, tag in pos_tag(tokens):  # returns token and corresponding tag from list of tokens
    lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
    #print(token, "=>", lemma)
    lemmatized_sentence.append(lemma)
  return ' '.join(lemmatized_sentence)

### Preprocessing input texts

In [ ]:
data_cleaned = [clean_text(text) for text in data]
data_cleaned[0] # data after preprocessing

'name david kochalski born small town called born may well hard working six children father mother small mill flour buckwheat prosperous comfortable went two schools one public school morning afternoon went religious school almost late night yes well raised spirit judaism school little city segregated catholics jews mind small town would say majority people small town jewish people inside town somehow dont know separated us jewish children catholic children know people poland catholic yes used friends yes felt maybe personally knew lot incidents whereby either small little would call separated words hardly got together incidents incidents pleasant incidents called house people regardless religious believe really religious people lovely family yes zionists city socialists communists even though time communists go underground religious institutions parents would say zionists religious aware going even though young youngster father used subscribe many newspapers jewish origin nonjewish pa

In [ ]:
data[0] # original data

' My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we were very hard working, six children, father and mother and we had a small mill, flour, buckwheat. We were not prosperous but comfortable.  I went to two schools. One was a public school in the morning. In the afternoon I went to a religious school until almost late at night.  Yes.  Well, I raised in the spirit of Judaism.  No, the school itself, in this little city, was segregated between Catholics and Jews. Mind you, it was a small town, and I would say the majority of the people in that small town were Jewish people. Inside the town, somehow, I don\'t know why, but they separated us Jewish children and Catholic children. As you know, most of the people in Poland were Catholic.  Yes, I used to have friends.  Yes, I did. I felt it, maybe not personally, but I knew of a lot of incidents whereby either they were small little -- I would call it -- we were separated, in other words, but ha

In [ ]:
data_cleaned = [lemmatize(texts) for texts in data_cleaned]
data_cleaned[0]

'name david kochalski bear small town call born may well hard work six child father mother small mill flour buckwheat prosperous comfortable go two school one public school morning afternoon go religious school almost late night yes well raise spirit judaism school little city segregate catholic jew mind small town would say majority people small town jewish people inside town somehow dont know separated u jewish child catholic child know people poland catholic yes use friend yes felt maybe personally knew lot incident whereby either small little would call separated word hardly get together incident incident pleasant incident call house people regardless religious believe really religious people lovely family yes zionist city socialists communist even though time communist go underground religious institution parent would say zionist religious aware go even though young youngster father use subscribe many newspaper jewish origin nonjewish paper paper nothing u however father brother r

## Creating the LDA model

In [ ]:
import gensim
import gensim.corpora as corpora

In [ ]:
def gen_words(texts):
    '''
      Input : lists of cleaned texts
      Output : list of words
    '''
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(data_cleaned)

print (data_words[0][0:20])

['name', 'david', 'kochalski', 'bear', 'small', 'town', 'call', 'born', 'may', 'well', 'hard', 'work', 'six', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat']


In [56]:
len(data_words[0])

5321

In [ ]:
id2word = corpora.Dictionary(data_words)
print(id2word)
print(id2word.token2id)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

Dictionary(41741 unique tokens: ['able', 'abroad', 'absorb', 'accord', 'across']...)
{'able': 0, 'abroad': 1, 'absorb': 2, 'accord': 3, 'across': 4, 'activity': 5, 'administrator': 6, 'adolf': 7, 'advance': 8, 'advantage': 9, 'afraid': 10, 'afternoon': 11, 'age': 12, 'ago': 13, 'agreement': 14, 'ahead': 15, 'aim': 16, 'air': 17, 'airman': 18, 'ajar': 19, 'alive': 20, 'allow': 21, 'almost': 22, 'already': 23, 'also': 24, 'always': 25, 'america': 26, 'american': 27, 'ammunition': 28, 'amongst': 29, 'amount': 30, 'amy': 31, 'angle': 32, 'angry': 33, 'animosity': 34, 'announced': 35, 'another': 36, 'answer': 37, 'anticipate': 38, 'anticipated': 39, 'anxious': 40, 'anybody': 41, 'anybodys': 42, 'anyhow': 43, 'anymore': 44, 'anything': 45, 'apartment': 46, 'apparatus': 47, 'appreciate': 48, 'apprentice': 49, 'approach': 50, 'approximately': 51, 'area': 52, 'arent': 53, 'arithmetic': 54, 'armament': 55, 'armistice': 56, 'army': 57, 'around': 58, 'arrange': 59, 'arreste': 60, 'artillery': 61, 

In [58]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 180 kB/s 
     |████████████████████████████████| 11.3 MB 60.2 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=5d14cdce3b70d30f862d2f46614641552e36de3ebe5c901617c3f02ebfe300e2
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [59]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.348179 -0.061696       1        1  29.002267
27     0.445643 -0.015147       2        1  22.671807
24     0.456626 -0.005116       3        1   9.045248
14     0.375287 -0.063279       4        1   8.729318
12     0.343578 -0.052124       5        1   8.071978
43     0.470725  0.038351       6        1   6.883049
17     0.442529 -0.004785       7        1   5.646649
6      0.413924 -0.018374       8        1   3.253007
19     0.428975  0.040937       9        1   3.023382
15     0.263895 -0.039611      10        1   1.314782
30     0.196560 -0.004868      11        1   0.615667
5      0.058132  0.123044      12        1   0.286290
21    -0.025105  0.121689      13        1   0.241499
35    -0.012877  0.338858      14        1   0.222283
34    -0.070368 -0.034393      15        1   0.169917
38     0.012368 -0.000379      16        1   0.135326
7     -0.042115  0.108956      17        1   0.132553
49    -0.121226 -0.023855      18        1   0.128187
33    -0.096001  0.030308      19        1   0.084436
16    -0.113372 -0.034881      20        1   0.067226
13    -0.125238 -0.030748      21        1   0.060086
37    -0.133113 -0.027963      22        1   0.059518
4     -0.101633 -0.011318      23        1   0.045431
2     -0.131006 -0.023620      24        1   0.033044
44    -0.133268 -0.027052      25        1   0.020667
48    -0.131644 -0.026643      26        1   0.019536
20    -0.129822 -0.017636      27        1   0.011482
23    -0.128632 -0.015953      28        1   0.009551
1     -0.126772 -0.013507      29        1   0.003830
42    -0.126297 -0.012881      30        1   0.002932
47    -0.125864 -0.012397      31        1   0.001739
28    -0.125869 -0.012452      32        1   0.001576
36    -0.125720 -0.012222      33        1   0.001353
9     -0.125401 -0.011846      34        1   0.000525
22    -0.125410 -0.011843      35        1   0.000512
40    -0.125303 -0.011706      36        1   0.000234
31    -0.125298 -0.011708      37        1   0.000232
25    -0.125304 -0.011706      38        1   0.000228
11    -0.125309 -0.011704      39        1   0.000223
41    -0.125308 -0.011705      40        1   0.000223
32    -0.125307 -0.011706      41        1   0.000223
0     -0.125316 -0.011703      42        1   0.000223
46    -0.125314 -0.011703      43        1   0.000222
39    -0.125313 -0.011703      44        1   0.000222
26    -0.125316 -0.011703      45        1   0.000222
8     -0.125313 -0.011701      46        1   0.000222
3     -0.125313 -0.011703      47        1   0.000221
29    -0.125314 -0.011703      48        1   0.000219
10    -0.125317 -0.011702      49        1   0.000219
45    -0.125322 -0.011700      50        1   0.000218, topic_info=           Term          Freq         Total Category  logprob  loglift
3872         uh  14384.000000  14384.000000  Default  30.0000  30.0000
397          go  23309.000000  23309.000000  Default  29.0000  29.0000
1060        yes   2828.000000   2828.000000  Default  28.0000  28.0000
527        know  19647.000000  19647.000000  Default  27.0000  27.0000
24         also   3030.000000   3030.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
28819        nu      0.000056     51.731892  Topic50 -10.6378  -0.7052
207       could      0.000056   4879.839405  Topic50 -10.6378  -5.2521
394        give      0.000056   4650.523171  Topic50 -10.6379  -5.2040
764    remember      0.000056   3873.945274  Topic50 -10.6379  -5.0213
553        live      0.000056   3639.013655  Topic50 -10.6379  -4.9588

[2999 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
40788     17  0.631679     abba
40788     23  0.210560     abba
7191       1  0.681318      abe
7191      10  0.278721      abe
7191      24  0.030969      abe
...      ...       ...      ...
106